In [ ]:
#Installing Pyspark
!pip3 install pyspark

     |████████████████████████████████| 204.2MB 56kB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=7e0f3aba88b3086d8caebdf888906881b2b3766ef07ae4b2af201d1dee375ae9
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


# Configuring the enviorment and importing the data 

In [ ]:
from pyspark import SparkConf, SparkContext
import collections #Do Python

In [ ]:
conf = SparkConf().setMaster("local").setAppName("olamundo")
sc = SparkContext(conf= conf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession 

In [ ]:
spark = SparkSession.builder.appName('primeira-versao').getOrCreate()

In [ ]:
df = spark.read.csv('/content/drive/My Drive/IMT/Pós Graduação/Big Data/Projeto/wta_matches_2020.csv', header = True, inferSchema=True)

In [ ]:
df.head(5)

[Row(tourney_id='2020-1049', tourney_name='Auckland', surface='Hard', draw_size=32, tourney_level='I', tourney_date=20200106, match_num=300, winner_id=200033, winner_seed=1, winner_entry=None, winner_name='Serena Williams', winner_hand='R', winner_ht=175, winner_ioc='USA', winner_age=38.2778918549, loser_id=202468, loser_seed=None, loser_entry=None, loser_name='Jessica Pegula', loser_hand='R', loser_ht=None, loser_ioc='USA', loser_age=25.864476386, score='6-3 6-4', best_of=3, round='F', minutes=95, w_ace=2, w_df=0, w_svpt=66, w_1stIn=43, w_1stWon=31, w_2ndWon=14, w_SvGms=10, w_bpSaved=4, w_bpFaced=5, l_ace=4, l_df=3, l_svpt=80, l_1stIn=50, l_1stWon=31, l_2ndWon=12, l_SvGms=9, l_bpSaved=12, l_bpFaced=15, winner_rank=10, winner_rank_points=3935, loser_rank=82, loser_rank_points=743),
 Row(tourney_id='2020-1049', tourney_name='Auckland', surface='Hard', draw_size=32, tourney_level='I', tourney_date=20200106, match_num=299, winner_id=200033, winner_seed=1, winner_entry=None, winner_name='S

In [ ]:
df.show()

+----------+------------+-------+---------+-------------+------------+---------+---------+-----------+------------+------------------+-----------+---------+----------+-------------+--------+----------+-----------+-------------------+----------+--------+---------+-------------+-----------+-------+-----+-------+-----+----+------+-------+--------+--------+-------+---------+---------+-----+----+------+-------+--------+--------+-------+---------+---------+-----------+------------------+----------+-----------------+
|tourney_id|tourney_name|surface|draw_size|tourney_level|tourney_date|match_num|winner_id|winner_seed|winner_entry|       winner_name|winner_hand|winner_ht|winner_ioc|   winner_age|loser_id|loser_seed|loser_entry|         loser_name|loser_hand|loser_ht|loser_ioc|    loser_age|      score|best_of|round|minutes|w_ace|w_df|w_svpt|w_1stIn|w_1stWon|w_2ndWon|w_SvGms|w_bpSaved|w_bpFaced|l_ace|l_df|l_svpt|l_1stIn|l_1stWon|l_2ndWon|l_SvGms|l_bpSaved|l_bpFaced|winner_rank|winner_rank_point

In [ ]:
df.printSchema()

# 2 Simple analysis

## What are the ages of the youngest and oldest players?

In [ ]:
# Analysing the age of players
df_age = df.select('winner_age','winner_name','winner_rank','loser_name','loser_age','loser_rank')
df_age.show()

+-------------+------------------+-----------+-------------------+-------------+----------+
|   winner_age|       winner_name|winner_rank|         loser_name|    loser_age|loser_rank|
+-------------+------------------+-----------+-------------------+-------------+----------+
|38.2778918549|   Serena Williams|         10|     Jessica Pegula| 25.864476386|        82|
|38.2778918549|   Serena Williams|         10|   Amanda Anisimova|18.3490759754|        25|
| 25.864476386|    Jessica Pegula|         82| Caroline Wozniacki|29.4893908282|        39|
|38.2778918549|   Serena Williams|         10|    Laura Siegemund|31.8412046543|        73|
|18.3490759754|  Amanda Anisimova|         25|   Eugenie Bouchard|25.8617385352|       262|
|29.4893908282|Caroline Wozniacki|         39|      Julia Goerges|31.1759069131|        41|
| 25.864476386|    Jessica Pegula|         82|       Alize Cornet| 29.954825462|        57|
|38.2778918549|   Serena Williams|         10|   Christina Mchale|27.6550308008|

In [ ]:
df_age.describe().show()

+-------+------------------+----------------+------------------+----------------+-----------------+------------------+
|summary|        winner_age|     winner_name|       winner_rank|      loser_name|        loser_age|        loser_rank|
+-------+------------------+----------------+------------------+----------------+-----------------+------------------+
|  count|               793|             797|               772|             797|              788|               742|
|   mean|25.290777534085205|            null|118.81994818652849|            null|25.42839373629742|  157.088948787062|
| stddev| 4.176552379723018|            null|166.08683043389414|            null|4.413103987509286|191.19066561928724|
|    min|     15.8165639973|Ajla Tomljanovic|                 1|Ajla Tomljanovic|    14.3299110198|                 1|
|    max|     38.3655030801|    Zarina Diyas|              1168|    Zarina Diyas|    39.7070499658|              1168|
+-------+------------------+----------------+---

In [ ]:
# Finding the 16 year old player 

filter_younger = df_age.filter(df['winner_age'] <= 16)
filter_older = df_age.filter(df['winner_age'] >= 38)

filter_younger.show()
filter_older.show()

+-------------+-----------+-----------+----------------+-------------+----------+
|   winner_age|winner_name|winner_rank|      loser_name|    loser_age|loser_rank|
+-------------+-----------+-----------+----------------+-------------+----------+
|15.8165639973| Cori Gauff|         67|Viktoria Kuzmova|21.6563997262|        61|
|15.8548939083| Cori Gauff|         67|     Naomi Osaka|22.2614647502|         4|
|15.8548939083| Cori Gauff|         67|  Venus Williams|39.5920602327|        55|
|15.8548939083| Cori Gauff|         67|  Sorana Cirstea| 29.787816564|        74|
+-------------+-----------+-----------+----------------+-------------+----------+

+-------------+---------------+-----------+------------------+-------------+----------+
|   winner_age|    winner_name|winner_rank|        loser_name|    loser_age|loser_rank|
+-------------+---------------+-----------+------------------+-------------+----------+
|38.2778918549|Serena Williams|         10|    Jessica Pegula| 25.864476386|   

## Who has the highest number of wins ans losses?

In [ ]:
df_wl = df.select('winner_name','loser_name')
df_wl.show()

+------------------+-------------------+
|       winner_name|         loser_name|
+------------------+-------------------+
|   Serena Williams|     Jessica Pegula|
|   Serena Williams|   Amanda Anisimova|
|    Jessica Pegula| Caroline Wozniacki|
|   Serena Williams|    Laura Siegemund|
|  Amanda Anisimova|   Eugenie Bouchard|
|Caroline Wozniacki|      Julia Goerges|
|    Jessica Pegula|       Alize Cornet|
|   Serena Williams|   Christina Mchale|
|   Laura Siegemund|         Cori Gauff|
|  Amanda Anisimova|    Darya Kasatkina|
|  Eugenie Bouchard|    Caroline Garcia|
|Caroline Wozniacki|       Lauren Davis|
|     Julia Goerges|Jil Belen Teichmann|
|    Jessica Pegula|    Tamara Zidansek|
|      Alize Cornet|       Petra Martic|
|   Serena Williams|      Camila Giorgi|
|  Christina Mchale|             Ann Li|
|        Cori Gauff|   Viktoria Kuzmova|
|   Laura Siegemund|       Caty Mcnally|
|  Amanda Anisimova|   Kateryna Kozlova|
+------------------+-------------------+
only showing top

In [ ]:
# Displays the amount of wins by players
df_winners = df_wl.groupBy('winner_name').count().orderBy('count', ascending=False).withColumnRenamed('count','Win_Amount')

df_winners.show()

+--------------------+----------+
|         winner_name|Win_Amount|
+--------------------+----------+
|      Elena Rybakina|        21|
|    Garbine Muguruza|        16|
|         Sofia Kenin|        15|
|          Ons Jabeur|        13|
|      Ashleigh Barty|        12|
|        Kiki Bertens|        12|
|     Aryna Sabalenka|        12|
|Ekaterina Alexand...|        12|
|     Anett Kontaveit|        12|
|       Petra Kvitova|        12|
|          Qiang Wang|        11|
|     Elina Svitolina|        11|
|       Elise Mertens|        11|
|        Saisai Zheng|        10|
|      Heather Watson|        10|
|        Simona Halep|        10|
|   Dayana Yastremska|        10|
|       Maria Sakkari|         9|
|       Magda Linette|         9|
|         Shuai Zhang|         9|
+--------------------+----------+
only showing top 20 rows



In [ ]:
# Displays the amount of losses by players
df_losers = df_wl.groupby('loser_name').count().orderBy('count', ascending = False).withColumnRenamed("count","Loss_Amount")

df_losers.show()

+--------------------+-----------+
|          loser_name|Loss_Amount|
+--------------------+-----------+
|     Tamara Zidansek|          8|
|Veronika Kudermetova|          7|
|      Belinda Bencic|          7|
|          Qiang Wang|          6|
|    Yulia Putintseva|          6|
|     Caroline Garcia|          6|
|    Christina Mchale|          6|
|          Yafan Wang|          6|
|  Katerina Siniakova|          6|
|             Lin Zhu|          6|
|     Darya Kasatkina|          6|
|     Elina Svitolina|          6|
|Anastasija Sevastova|          6|
|     Nina Stojanovic|          6|
|       Maria Sakkari|          6|
|    Viktoria Kuzmova|          5|
|         Donna Vekic|          5|
| Kristina Mladenovic|          5|
|     Aryna Sabalenka|          5|
|     Anett Kontaveit|          5|
+--------------------+-----------+
only showing top 20 rows

